**Upload the Dataset to Python**






In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents (2).json to intents (2) (2).json


In [ ]:
import json

with open('/content/intents (2).json') as file:
    intents = json.load(file, strict = False) e '/n' (this is very minor)
intents = intents['intents'] # Get all of the individual intents from our dataset

5. Run the code below to see what our dataset looks like right now. (The code just prints out the dataset nicely so that it is readable)

In [ ]:
print("[", end = "")
for intent in intents:
  print("{", end = "")
  for key, value in intent.items():
    print("{}: {},".format(key, value))
  print("\b\b\n},")
print("\b\b]")

[{tag: greeting,
patterns: ['Hi', 'Hello', 'Hey', 'Hi there', 'Good morning', 'Good afternoon', 'Good evening'],
responses: ['Hello! How can I assist you today?', 'Hi there! What can I help you with?', 'Hey! How can I be of service?'],

},
{tag: balance_inquiry,
patterns: ["What's my balance?", 'Can you tell me my balance?', 'How much money do I have?', "What's my account balance?", 'my account balance'],
responses: ['To check your balance, please enter your account number and PIN.', 'You can check your balance by logging into your account on our website or mobile app.', 'Your current balance is $XXX.XX.'],

},
{tag: withdrawal,
patterns: ['How do I withdraw money?', 'Can I withdraw cash from this ATM?', 'How much money can I withdraw?', "What's the maximum withdrawal limit?"],
responses: ['To withdraw money, please insert your card and enter your PIN. Then select the amount you wish to withdraw.', 'Yes, you can withdraw cash from this ATM. Please insert your card and follow the on

# **Import the Libraries**

Here, we install & import all of the remaining libraries that we will need to use (we already imported json to read in our dataset, and we will be importing nltk for natural language processing in the next step). 

In [ ]:
#tflearn requires us to install an earlier version of tensorflow
!pip install tensorflow==1.15
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==1.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf

import tflearn
import random
import pickle

# **Natural Language Processing**
Our final goal with this project is to train our chatbot based on the intents dataset to recognize which intent a users statement might fall under. The chatbot would then respond accordingly with a response from the given intent. 

In order to accomplish this, we first need to process our intents dataset using **Natural Language Processing** or NLP. This allows us to simplify the dataset to make it more easy to understand for our chatbot. The first thing we will need to do is download the required libraries for NLP. We will be using a library called the Natural Language Toolkit or NLTK.

In [ ]:
import nltk
nltk.download('all')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

Now we can get into the specifics of the NLP. This is a long block of code which I cannot break up, so I will write comments to explain the code. Look out for any green text (these are the comments).

In [ ]:

retrain_model = True

if retrain_model:
    all_words = [] #This will be a list of all the words used in any of the 'patterns' in each intent
    all_tags = [] #This will be a list of all the 'tag's associated with the intents
    intent_patterns = [] #This will be a list containing all of the 'patterns' for each intent where each individual pattern is grouped together
    intent_tags = [] #This will be a list correlated with 'intent_patterns' where every pattern in 'intent_patterns' is correlated with its respective
                     #Tag in this list
    
    #Here we fill in all of the lists above. Note that we tokenize the words in each pattern which means we split each pattern into individual words
    for intent in intents:
        for pattern in intent['patterns']:
            words = nltk.word_tokenize(pattern)

            all_words.extend(words)
            intent_patterns.append(words)
            intent_tags.append(intent['tag'])
            
        all_tags.append(intent['tag'])
      
    #Here we stem the words in all_words. This means that we reduce every word down to its root form or stem. This will prevent our chatbot from confusin
  
    all_words = [stemmer.stem(word.lower()) for word in all_words]
    all_words = sorted(list(set(all_words)))
    
    all_tags = sorted(all_tags)
    
    x_train = []
    y_train = []
    
    y_empty = [0 for i in range(len(all_tags))]
    
    #Here we are creating our training set input and output values for our deep learning algorithm
    #We will do this by iterating through our intents and turning each one into a bag of words, or a vector that indicates which words are in each pattern.
    #These bags of words will be the x values and the y values will be the intent that each bag of words is associated with.
    #The machine learning will train on this data and will be able to determine which bag of words its corresponding intent. 
    for index, intent in enumerate(intent_patterns):
        bag_of_words = []
        
        intent_words = [stemmer.stem(word.lower()) for word in intent]
        
        for word in all_words:
            if word in intent_words:
                bag_of_words.append(1)
            else:
                bag_of_words.append(0)
                
        one_hot_encode_y = y_empty[:]
        one_hot_encode_y[all_tags.index(intent_tags[index])] = 1
        
        x_train.append(bag_of_words)
        y_train.append(one_hot_encode_y)
    
    #Here is the data we will be using to train our neural network later
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    #Here we just save our training data so we don't need to process it again if we just want to run our chatbot
    with open('training_data.pickle', 'wb') as f:
        pickle.dump((all_words, all_tags, x_train, y_train), f)
else:
    with open('training_data.pickle', 'rb') as f:
        all_words, all_tags, x_train, y_train = pickle.load(f)

# **Using Deep Learning**
 Below, we create a neural network with an input layer that takes in these bags of words, two hidden layers, and an output layer that gives a probability for the bag of words being correlated with each intent.

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
tf.reset_default_graph()

#Create the neural network layers
neural_net = tflearn.input_data(shape = [None, len(x_train[0])])
neural_net = tflearn.fully_connected(neural_net, 8)
neural_net = tflearn.fully_connected(neural_net, 8)
#Here we use the softmax activation function so the output of our neural network is a probability. We will make use of this later
neural_net = tflearn.fully_connected(neural_net, len(y_train[0]), activation = 'softmax')
neural_net = tflearn.regression(neural_net)

model = tflearn.DNN(neural_net)

Now that we have created our neural network, we can train it and save our model for later.

In [ ]:
from keras.callbacks import TensorBoard, EarlyStopping
earlyStopping = EarlyStopping(monitor = 'loss', patience = 400, mode = 'min', restore_best_weights = True)

In [ ]:
 
if retrain_model:
    #Here we train the neural network with the training data we created in the NLP stage
    history_training = model.fit(x_train, y_train, n_epoch = 400, batch_size = 32, show_metric = True)
    model.save('model.tfl')
else:
    model.load('./model.tfl')

Training Step: 1599  | total loss: 1.99763 | time: 0.025s
| Adam | epoch: 400 | loss: 1.99763 - acc: 0.8424 -- iter: 096/110
Training Step: 1600  | total loss: 1.81983 | time: 0.037s
| Adam | epoch: 400 | loss: 1.81983 - acc: 0.8582 -- iter: 110/110
--


In [ ]:
import random

# account details
account_details = {
    "name": "John Doe",
    "account_number": "1234567890",
    "balance": 10000,
    "pin": "1234"
}

In [ ]:
# function to change ATM pin
def change_pin():
    print("Please enter your current ATM pin:")
    current_pin = input()
    if current_pin != account_details["pin"]:
        print("Invalid pin. Please try again.")
        return
    print("Please enter your new ATM pin:")
    new_pin = input()
    print("Please confirm your new ATM pin:")
    confirm_pin = input()
    if new_pin != confirm_pin:
        print("New pin and confirm pin do not match. Please try again.")
        return
    account_details["pin"] = new_pin
    print("ATM pin changed successfully.")
  



In [ ]:
# function for balance inquiry
def bal_inquiry():
    print("Please enter your Account Number:")
    acc_no = input()
    if acc_no != account_details["account_number"]:
        print("Invalid Account Number. Please try again.")
        return

    print("Account Balance =")
    print(account_details["balance"])

# **Creating the Chatbot**
.

Below, we create the first step of our process. The first thing we need to do is convert the users input to a bag of words. That's exactly what the function below does.

In [ ]:
def text_to_bag(text, all_words):
    #Initialize the bag of words by creating an empty slot for every word in the vector
    bag_of_words = [0 for i in range(len(all_words))]
    
    #First we split up the input into individual words and stem them so they match the same format as in our vector
    text_words = nltk.word_tokenize(text)
    text_words = [stemmer.stem(word.lower()) for word in text_words]
    
    #Now we create the bag of words by filling in a 1 for the words that the user used
    for word in text_words:
        if word in all_words:
            bag_of_words[all_words.index(word)] = 1
    
    #And return the bag of words
    return np.array(bag_of_words)

This function will get the users input, call the bag_of_words function to turn it into a bag of words, and then pass the bag of words into the neural network to get a prediction. Finally, it will print out the chatbots response.

In [ ]:
def chat():
    #Starting message
    print("Enter a message to talk to the bot [type quit to exit].")
    
    #Reset the context state since there is no context at the beginning of the conversation
    context_state = None
    
    #This is what the bot will say if it doesn't understand what the user is saying
    default_responses = ['Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!',

                         'I dont understand that! Try rephrasing or saying something else.']

    #This chat loop will go on forever until the user types quit
    while True:
        user_chat = str(input('You: '))
        if user_chat.lower() == 'quit':
            break
        
        #Convert chat to bag of words
        user_chat_bag = text_to_bag(user_chat, all_words)

        #Pass bag of words into our neural network
        response = model.predict([user_chat_bag])[0]

        #Get the intent that the bag of words is most highly correlated with
        response_index = np.argmax(response)
        response_tag = all_tags[response_index]
        
        #If the neural network is fairly certain that it has chosen the right intent (and isnt randomly guessing)
        #In this case, we will only get a response if the neural network is more than 80% certain
        if response[response_index] > 0.8:
            for intent in intents:
                #Get the intent that is predicted
                if intent['tag'] == response_tag:
                    #Check if this response is associated with a specific context
                    if response_tag =="ATM Pin":
                      change_pin()
                      break
                    if response_tag =="balance_inquiry":
                      bal_inquiry()
                      break

                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context_state:
                        #Get all of the possible responses from this intent
                        possible_responses = intent['responses']
                        #If this intent is associated with a context set, then set the context state
                        if 'context_set' in intent:
                            context_state = intent['context_set']
                        else:
                            context_state = None
                        #Select a random message from the intent responses
                        print(random.choice(possible_responses))
                    else:
                        #Print a did not understand message
                        print(random.choice(default_responses))
        else:
            #Print a did not understand message
            print(random.choice(default_responses))

Now our chatbot is complete and we can actually talk to it! Run the code below to talk to the chatbot. 

In [ ]:
chat()

Enter a message to talk to the bot [type quit to exit].
Hey! How can I be of service?
Please enter your current ATM pin:
Please enter your new ATM pin:
Please confirm your new ATM pin:
ATM pin changed successfully.
Please enter your Account Number:
Account Balance =
10000
Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!
To withdraw money, please insert your card and enter your PIN. Then select the amount you wish to withdraw.
I dont understand that! Try rephrasing or saying something else.
Don't worry, we'll get this sorted out. Please provide me with more information about the issue.
You can contact at: NUMBER


As you can see, our chatbot works pretty well with the intents in our dataset. 

